# ❄️ Snowflake Chat with NCBI PubMed Articles ❄️

In this Hands on Lab we will load public NCBI Pubmed articles into snowflake and set up Cortex Search Service on the articles so we can leverage Cortex LLMs to asks questions in natural laguage about the articles.

DO NOT RUN ALL cells, we will step throught each cell one at a time. 

Adapted version of this quickstart: https://quickstarts.snowflake.com/guide/ask_questions_to_your_own_documents_with_snowflake_cortex_search/index.html?index=..%2F..index#0 



1. Search and add snowflake.core and langchain from Packages Dropdown on the top right corner

2. Start the Notebook session

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
st.image( "https://quickstarts.snowflake.com/guide/ask_questions_to_your_own_documents_with_snowflake_cortex_search/img/1d96fe59a89a3ac5.png", width=1000)

In [ ]:
/**************************
 Set the context - update the database
**************************/
use database fh_pmc_data1;
use schema pmc_oa_opendata;

In [ ]:

--https://www.ncbi.nlm.nih.gov/pmc/tools/pmcaws/
/**************************
Pubmed article TEXT files are available in a public s3
Associated metadata about the articles available are in a metadata file
**************************/
create or replace stage pmc_oa_comm_raw url = 's3://pmc-oa-opendata/oa_comm';

create 
or replace file format my_csv_format type = csv skip_header=1;


--Key,ETag,Article Citation,AccessionID,Last Updated UTC (YYYY-MM-DD HH:MM:SS),PMID,License,Retracted

CREATE 
or replace TABLE oa_comm_metadata (
Key text,
etag text,
article_citation text,
accessionid text,
last_updated_utc  text ,
pmid text, 
license text,
retracted text
);


In [ ]:
/**************************
Load the metadata file into snowflake table
**************************/

copy into oa_comm_metadata FROM @pmc_oa_comm_raw/txt/metadata/csv/oa_comm.filelist.csv 
file_format= 'my_csv_format'
ON_ERROR= 'CONTINUE';

select * from oa_comm_metadata order by accessionid asc limit 20;



In [ ]:
/**************************
Snowpark Python function to chunk files, chunk size configurable
**************************/

create or replace function text_chunker(file_url string)
returns table (chunk varchar)
language python
runtime_version = '3.9'
handler = 'text_chunker'
packages = ('snowflake-snowpark-python','langchain')
as
$$
from snowflake.snowpark.types import StringType, StructField, StructType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from snowflake.snowpark.files import SnowflakeFile
import io
import logging
import pandas as pd

class text_chunker:

    def read_txt(self, file_url: str) -> str:
    
        logger = logging.getLogger("udf_logger")
        logger.info(f"Opening file {file_url}")
    
        with SnowflakeFile.open(file_url, 'rb') as f:
            buffer = io.BytesIO(f.readall())

        try:
            text = buffer.getvalue().decode('utf-8').replace('\n', ' ').replace('\0', ' ')

        except Exception as e:
            logger.warn(f"Unable to extract from file {file_url}: {str(e)}")
            text = "Unable to Extract"
        
        return text

    def process(self, file_url: str):

        text = self.read_txt(file_url)
        
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size = 4000, #Adjust this as you see fit
            chunk_overlap  = 400, #This let's text have some form of overlap. Useful for keeping chunks contextual
            length_function = len
        )
    
        chunks = text_splitter.split_text(text)
        df = pd.DataFrame(chunks, columns=['chunk'])
        
        yield from df.itertuples(index=False, name=None)
$$;


Note that if you have PDF documents you can alternatively use Cortex AI Task Functions [Parse Document](https://docs.snowflake.com/en/user-guide/snowflake-cortex/parse-document) and [Split Text Recursive Character](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex). Instead of writing a chunking snowpark function you can call these functions on your staged pdf docs. 

In [ ]:
--Table to load text from pmc articles in AWS external stage 
create or replace TABLE PMC_OA_CHUNKS_TABLE ( 
    RELATIVE_PATH VARCHAR(16777216), -- Relative path to the file
    ABS_PATH  VARCHAR(16777216), -- Path for the file
    Etag VARCHAR(16777216),
    SCOPED_FILE_URL VARCHAR(16777216), -- Scoped url (you can choose which one to keep depending on your use case)
    CHUNK VARCHAR(16777216)-- Piece of text
);


In [ ]:
/**************************
Let's see the text_chunker function in action 
This is what we will insert in the chunks table
**************************/

with oa_comm as (select array_to_string(array_slice(split(key, '/'),1,4), '/') as relative_path , etag 
from oa_comm_metadata where etag not in (select etag from PMC_OA_chunks_table ) limit 10)
select 
    relative_path, 
    get_absolute_path(@pmc_oa_comm_raw, relative_path) as abs_path,
    etag,
    build_scoped_file_url(@pmc_oa_comm_raw, relative_path) as scoped_file_url,
    func.chunk as chunk
from 
    oa_comm,
    TABLE(text_chunker(build_scoped_file_url(@pmc_oa_comm_raw, relative_path))) as func

In [ ]:
/**************************
Loads Articles that have not been processed yet into PMC_OA_CHUNKS_TABLE
Batch processing 100 articles by using "LIMIT 100"
**************************/

insert into PMC_OA_chunks_table (relative_path, abs_path,etag,scoped_file_url, chunk)
with oa_comm as 
(select array_to_string(array_slice(split(key, '/'),1,4), '/') as relative_path , etag 
from oa_comm_metadata where etag not in (select etag from PMC_OA_chunks_table ) limit 100)
select 
    relative_path, 
    get_absolute_path(@pmc_oa_comm_raw, relative_path) as abs_path,
    etag,
    build_scoped_file_url(@pmc_oa_comm_raw, relative_path) as scoped_file_url,
    func.chunk as chunk
from 
    oa_comm,
    TABLE(text_chunker(build_scoped_file_url(@pmc_oa_comm_raw, relative_path))) as func;


In [ ]:
/**************************
Create a single view that combines the article metadata and the chunked text
**************************/
--View for Search service definition
create or replace view pmc_service_vw as 
(select pmc_oa_chunks_table.etag, article_citation,accessionid, last_updated_utc, pmid, retracted, license, chunk  from pmc_oa_opendata.PMC_OA_chunks_table 
left join  pmc_oa_opendata.oa_comm_metadata on pmc_oa_chunks_table.etag=oa_comm_metadata.etag);


In [ ]:
---Pick PMC ID to chat with.

Select * from pmc_service_vw limit 10;


[Create Cortex Search Documentation](https://docs.snowflake.com/en/sql-reference/sql/create-cortex-search)

[Embedding models available](https://docs.snowflake.com/en/user-guide/snowflake-cortex/cortex-search/cortex-search-overview#regional-availability)

In [ ]:
/**************************
Create a search service 

--UPDATE WAREHOUSE 
**************************/
Create or replace cortex search service my_pmc_search_service on chunk 
attributes etag, article_citation, accessionid, last_updated_utc, pmid, retracted, license 
warehouse= fh_pmc_wh1
target_lag= '1 min'
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
as (
select etag, article_citation,accessionid, last_updated_utc, pmid, retracted, license, chunk from pmc_oa_opendata.pmc_service_vw
);


In [ ]:
Describe Cortex Search service my_pmc_search_service;

**How to Query the Cortex Search Service**

In [ ]:
## Import python packages
##UPDATE DATABASE Line 11

import streamlit as st
import pandas as pd 
from snowflake.core import Root
import json
import pandas as pd

root = Root(session)
pmc_search_service = (root
  .databases["FH_PMC_DATA1"]
  .schemas["PMC_OA_OPENDATA"]
  .cortex_search_services["my_pmc_search_service"]
)

In [ ]:
#Pick an ID from the search service view
#https://www.ncbi.nlm.nih.gov/pmc/?term=open%20access%5Bfilter%5D

similar_articles=pmc_search_service.search(
    query='article abstract',
    columns= ["CHUNK", "ACCESSIONID"],
    filter={"@eq": {"accessionid": "PMC9975415"} },
    limit=10
)

In [ ]:
similar_articles_resp=similar_articles.to_json()
data = json.loads(similar_articles_resp)
df = pd.json_normalize(data['results'])
df

In [ ]:
#example streamlit code 
#UPDATE DATABASE LINE 16
import streamlit as st # Import python packages
from snowflake.snowpark.context import get_active_session
session = get_active_session() # Get the current credentials

import pandas as pd

pd.set_option("max_colwidth",None)
num_chunks = 3 # Num-chunks provided as context. Play with this to check how it affects your accuracy

def create_prompt (myquestion, rag, article_chosen=None):
    st.write(article_chosen)
    root = Root(session)
    pmc_search_service = (root
                .databases["FH_PMC_DATA1"]
                .schemas["PMC_OA_OPENDATA"]
                .cortex_search_services["my_pmc_search_service"]
                )
    if rag == 1:  
        if article_chosen:
            similar_articles=pmc_search_service.search(
                query='article abstract',
                columns= ["CHUNK", "ACCESSIONID"],
                filter={"@eq": {"accessionid": f"{article_chosen}" }},
                limit=10
                )
            
            similar_articles_resp=similar_articles.to_json()
            data = json.loads(similar_articles_resp)
            df_context = pd.json_normalize(data['results'])
            st.write(df_context)
        
            
            context_lenght = len(df_context) -1
    
            prompt_context = ""
            for i in range (0, context_lenght):
                prompt_context += df_context._get_value(i, 'CHUNK')
    
            prompt_context = prompt_context.replace("'", "")
            accessionid =  df_context._get_value(0,'ACCESSIONID')
        
            prompt = f"""
              'You are an expert assistance extracting information from context provided. 
               Answer the question based on the context. Be concise and do not hallucinate. 
               If you don´t have the information just say so.
              Context: {prompt_context}
              Question:  
               {myquestion} 
               Answer: '
               """
        else:
            similar_articles=pmc_search_service.search(
                query='article abstract',
                columns= ["CHUNK", "ACCESSIONID"],
                limit=10
                )
            
            similar_articles_resp=similar_articles.to_json()
            data = json.loads(similar_articles_resp)
            df_context = pd.json_normalize(data['results'])
        
            
            context_lenght = len(df_context) -1
    
            prompt_context = ""
            for i in range (0, context_lenght):
                prompt_context += df_context._get_value(i, 'CHUNK')
    
            prompt_context = prompt_context.replace("'", "")
            accessionid =  df_context._get_value(0,'ACCESSIONID')
        
            prompt = f"""
              'You are an expert assistance extracting information from context provided. 
               Answer the question based on the context. Be concise and do not hallucinate. 
               If you don´t have the information just say so.
              Context: {prompt_context}
              Question:  
               {myquestion} 
               Answer: '
               """

    else:
        prompt = f"""
         'Question:  
           {myquestion} 
           Answer: '
           """
        accessionid = "None"
        
        
    return prompt,accessionid

def complete(myquestion, model_name, rag = 1, article_chosen=None):
    #st.write(article_chosen)

    prompt,accessionid =create_prompt (myquestion, rag, article_chosen)
    cmd = f"""
             select SNOWFLAKE.CORTEX.COMPLETE(?,?) as response
           """
    
    df_response = session.sql(cmd, params=[model_name, prompt]).collect()
    return df_response, accessionid

def display_response (question, model, rag=0, article_chosen=None):
    response, accessionid= complete(question, model, rag, article_chosen)
    res_text = response[0].RESPONSE
    st.markdown(res_text)
    if rag == 1:
        article_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{accessionid}/"
        text= f"\n\nAssociated NCBI AccessionID that may be useful:\n[{accessionid}]({article_url})"
        st.write(text)

#Main code

st.title("Chat with NCBI Pubmed Articles with Cortex LLMs:")
st.write("""You can ask questions and decide if you want to the NCBI Articles for context or allow the model to create their own response.""")

docs_available = session.sql("select distinct ACCESSIONID from PMC_SERVICE_VW limit 10").collect()
list_docs = []
for doc in docs_available:
    list_docs.append(doc["ACCESSIONID"])

rag = st.checkbox('Use articles as context?')

if rag:
    use_rag = 1
    article_chosen=st.selectbox("Choose an Article for Context",list_docs)
else:
    use_rag = 0
    article_chosen=None

#Here you can choose what LLM to use. Please note that they will have different cost & performance
model = st.selectbox('Select your model:',('claude-3-5-sonnet',
                                    'mixtral-8x7b',
                                    'snowflake-arctic',
                                    'mistral-large',
                                    'llama3-8b',
                                    'llama3-70b',
                                    'reka-flash',
                                     'mistral-7b',
                                     'llama2-70b-chat',
                                     'gemma-7b'))

question = st.text_input("Enter question", placeholder="Is there articles related to type-II diabetes?", label_visibility="collapsed")

if question:
    display_response (question, model, use_rag, article_chosen)